# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [23]:
!pip install requests beautifulsoup4 #pip install requests beautifulsoup4
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin

BASE_URL = "http://books.toscrape.com/" # Base URL for the book store

Defaulting to user installation because normal site-packages is not writeable


ERROR: Invalid requirement: '#pip': Expected package name at the start of dependency specifier
    #pip
    ^


In [27]:
# Send a GET request to the base URL
res = requests.get(BASE_URL)

# Mapping of text to numerical rating
RATING_MAP = {
    'One': 1,
    'Two': 2,
    'Three': 3,
    'Four': 4,
    'Five': 5
}

def get_genre(book_url):
    """Get genre from the breadcrumb on the book detail page."""
    res = requests.get(book_url)
    soup = BeautifulSoup(res.text, 'html.parser')
    breadcrumb = soup.select('ul.breadcrumb li a')
    if len(breadcrumb) >= 3:
        return breadcrumb[2].text.strip()
    return None

def get_description(soup):
    """Extract the product description if available."""
    desc = soup.select_one('#product_description')
    if desc:
        desc_tag = desc.find_next_sibling('p')
        return desc_tag.text.strip() if desc_tag else None
    return None

def scrape_books(min_rating=4, max_price=20.0):
    books_data = []
    next_page = "catalogue/page-1.html"
    
    while next_page:
        res = requests.get(urljoin(BASE_URL, next_page))
        soup = BeautifulSoup(res.text, 'html.parser')
        books = soup.select('article.product_pod')

        for book in books:
            # Get title and relative detail URL
            title = book.h3.a['title']
            detail_url = urljoin(BASE_URL, book.h3.a['href'].replace('../../../', 'catalogue/'))
            book_page = requests.get(detail_url)
            book_soup = BeautifulSoup(book_page.text, 'html.parser')

            # Get UPC from the product table
            upc_tag = book_soup.select_one('table.table.table-striped tr:nth-of-type(1) td')
            upc = upc_tag.text.strip() if upc_tag else None

            # Price
            import re
            price_str = book.select_one('p.price_color').text.strip()
            price_num = re.search(r'[\d\.]+', price_str.replace('£', ''))
            price = float(price_num.group()) if price_num else 0.0

            # Rating
            rating_class = book.select_one('p.star-rating')['class'][1]
            rating = RATING_MAP.get(rating_class, 0)

            # Filter
            if rating < min_rating or price > max_price:
                continue

            # Availability
            availability = book.select_one('p.instock.availability').text.strip()

            # Genre
            genre = get_genre(detail_url)

            # Description
            description = get_description(book_soup)

            books_data.append({
                'UPC': upc,
                'Title': title,
                'Price (£)': price,
                'Rating': rating,
                'Genre': genre,
                'Availability': availability,
                'Description': description
            })

        # Pagination
        next_btn = soup.select_one('li.next a')
        next_page = urljoin(next_page, next_btn['href']) if next_btn else None

    return pd.DataFrame(books_data)

if __name__ == "__main__":
    df = scrape_books(min_rating=4, max_price=20.0)
    print(df.head())         # Print the first few rows to verify
    print(df.shape)          # Show how many rows were collected



    UPC                                              Title  Price (£)  Rating  \
0  None                                        Set Me Free      17.46       5   
1  None  The Four Agreements: A Practical Guide to Pers...      17.66       5   
2  None                                     Sophie's World      15.94       5   
3  None            Untitled Collection: Sabbath Poems 2014      14.27       4   
4  None                                    This One Summer      19.49       4   

  Genre Availability Description  
0  None     In stock        None  
1  None     In stock        None  
2  None     In stock        None  
3  None     In stock        None  
4  None     In stock        None  
(75, 7)
